In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split 
import tensorflow as tf 
from tensorflow.keras.layers import Input , GlobalAveragePooling2D , Dense , Dropout 

In [ ]:
train_data = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_data = pd.read_csv('test_features.csv')


In [ ]:
fig = plt.figure(figsize=(10, 7))
for i in range(10) : 
    fig.add_subplot(2, 2, 1)
    img = mpimg.imread(train_data['filepath'][i])
    imgplot = plt.imshow(img)
    plt.show()

In [ ]:
for column in train_labels.columns[1:] : 
    sumtion = sum(train_labels[column])
    print(f'the class : {column} , the count : {sumtion} ')

In [ ]:
train_labels = train_labels.drop('id' , axis = 1)


In [ ]:
label_columns = train_labels.columns 

In [ ]:
train_labels['label_int'] = train_labels.columns.get_indexer(train_labels.idxmax(1))

In [ ]:
train_labels['label'] = label_columns[train_labels['label_int']]

In [ ]:
train_labeled_data = pd.concat([train_data , train_labels['label'] ] ,axis=1)

In [ ]:
X_train , X_val  = train_test_split(
    train_labeled_data , test_size= .15, shuffle= True , random_state= 123)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest' , 
        preprocessing_function = tf.keras.applications.efficientnet_v2.preprocess_input)

In [ ]:
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
preprocessing_function = tf.keras.applications.efficientnet_v2.preprocess_input )


In [ ]:
train_ds = train_generator.flow_from_dataframe(X_train  , x_col='filepath' , y_col='label' ,
    target_size=(224 ,224)
)

In [ ]:
val_ds = val_generator.flow_from_dataframe(X_val , x_col = 'filepath' , y_col = 'label' , target_size=(224 ,224))

In [ ]:
pretrained_model = tf.keras.applications.EfficientNetB0(include_top= False ,
    input_shape  = (224,224,3) , weights='imagenet' , 
)

In [ ]:
pretrained_model.trainable = False 

In [ ]:
input = Input(shape = (224,224,3))
X = pretrained_model(input )
X = GlobalAveragePooling2D()(X)
X = Dense(1024, activation='relu')(X)
X = Dropout(.5)(X)
X = Dense(8 ,  activation='softmax')(X)
model = tf.keras.Model(input , X) 

In [ ]:
model.compile(optimizer='adam' , loss= 'categorical_crossentropy' , metrics='accuracy')


In [ ]:
history = model.fit_generator(train_ds , epochs= 5 , validation_data= val_ds) 